In [1]:
import requests
import xml.etree.cElementTree as ET
import pandas as pd

In [2]:
# URL dari data XML
url = 'https://data.bmkg.go.id/DataMKG/MEWS/DigitalForecast/DigitalForecast-SulawesiTengah.xml'

In [3]:
# Fungsi untuk mengunduh dan mengurai data XML
def get_weather_data(url):
    response = requests.get(url)
    response.raise_for_status()  # Memeriksa apakah permintaan berhasil
    xml_data = response.content

    # Parse XML
    tree = ET.ElementTree(ET.fromstring(xml_data))
    root = tree.getroot()

    # Data yang akan diambil
    data = []

    # Menjelajahi elemen di dalam XML
    for area in root.findall('forecast/area'):
        area_name = area.get('description')
        area_id = area.get('id')
        
        for parameter in area.findall('parameter'):
            parameter_id = parameter.get('id')
            parameter_description = parameter.get('description')
            
            for timerange in parameter.findall('timerange'):
                datetime = timerange.get('datetime')
                
                for value in timerange.findall('value'):
                    value_type = value.get('type')
                    value_text = value.text
                    
                    data.append({
                        'area_id': area_id,
                        'area_name': area_name,
                        'parameter_id': parameter_id,
                        'parameter_description': parameter_description,
                        'datetime': datetime,
                        'value_type': value_type,
                        'value_text': value_text
                    })
                    
    return data

In [4]:
# Fungsi untuk menyimpan data ke CSV
def save_to_csv(data, filename):
    df = pd.DataFrame(data)
    df.to_csv(filename, index=False)

In [5]:
# Mengambil data prakiraan cuaca
weather_data = get_weather_data(url)

In [6]:
# Menampilkan data dalam bentuk tabel
df_weather = pd.DataFrame(weather_data)
print(df_weather.head())

  area_id area_name parameter_id parameter_description      datetime  \
0  501520    Ampana           hu              Humidity  202406180000   
1  501520    Ampana           hu              Humidity  202406180600   
2  501520    Ampana           hu              Humidity  202406181200   
3  501520    Ampana           hu              Humidity  202406181800   
4  501520    Ampana           hu              Humidity  202406190000   

  value_type value_text  
0       None         90  
1       None         80  
2       None         95  
3       None         90  
4       None         90  


In [7]:
# Menyimpan data ke dalam file CSV
save_to_csv(weather_data, 'prakiraan_cuaca_sulawesi_tengah.csv')